In [2]:
import pandas as pd
import polars as pl

In [3]:
df = pd.read_parquet('data/output/chuncks/chunk_0_4999.parquet')

In [5]:
df

,allele,main_allele,position
B205,TT,GG,1384
IRIS_313-11532,AA,CC,1399
CX349,TT,CC,1409
B161,AT,AA,1427
B188,AT,AA,1427
...,...,...,...
IRIS_313-11477,AA,CC,150280
IRIS_313-11737,AA,CC,150280
IRIS_313-8822,AA,CC,150280
IRIS_313-9368,AA,CC,150280


In [74]:
df2 = pl.read_parquet('data/output/all_main_allele.parquet')
df2 = df2.sort('position')
df2

position,main_allele
i64,str
1384,"""GG"""
1399,"""CC"""
1409,"""CC"""
1427,"""AA"""
1429,"""CC"""
…,…
43269742,"""GG"""
43269856,"""AA"""
43269879,"""AA"""


In [66]:
df2["position"].diff().describe().round(2)

count    1107867.00
mean          39.06
std          655.61
min            1.00
25%            5.00
50%           12.00
75%           29.00
max       432274.00
Name: position, dtype: float64

In [68]:
desc = df2[100:250]["position"].describe()
start = int(desc["min"])
end = int(desc["max"])
print(f"Start: {start}, End: {end}")

Start: 1974, End: 4732


In [69]:
df3 = pl.read_parquet('data/output/SNPs/B001.parquet').to_pandas()

In [70]:
df3[(df3["position"] >= start) & (df3["position"] <= end)]

,allele,main_allele,position,individual


In [78]:
df2[100:250]

position,main_allele
i64,str
1974,"""GG"""
1977,"""GG"""
1979,"""TT"""
1986,"""AA"""
1989,"""GG"""
…,…
4629,"""TT"""
4658,"""TT"""
4675,"""GG"""


,allele,main_allele,position,individual
0,GT,GG,6918,B001
1,GT,GG,6946,B001
2,CC,AA,17263,B001
3,AG,AA,21665,B001
4,TG,TT,21680,B001
...,...,...,...,...
43797,TT,CC,43244177,B001
43798,AA,AC,43253999,B001
43799,AA,GG,43254132,B001
43800,GG,GA,43254300,B001


In [37]:
ref_df = pl.read_parquet("data/output/all_main_alleles.parquet")
ref_df = ref_df.drop(["__index_level_0__"])
individual_df = pl.read_parquet("data/output/SNPs/B001.parquet")

In [ ]:
sub_ref = ref_df[0:5000]
start_pos = sub_ref["position"][0]
end_pos = sub_ref["position"][-1]
print(f"Start: {start_pos}, End: {end_pos}")
# Filter the individual_df DataFrame to get the subset of individuals



Start: 1384, End: 152418


In [ ]:
sub_individual = individual_df.filter(
    (individual_df["position"] >= start_pos) & (individual_df["position"] <= end_pos)
)

sub_ref_updated = sub_ref.join(
    sub_individual[["allele", "position"]],
    on="position",
    how="left"
)

sub_ref_updated = sub_ref_updated.with_columns(
    pl.col("allele").fill_null(pl.col("main_allele"))
)

sub_ref_updated.filter(
    (pl.col("main_allele") != pl.col("allele"))
)

position,main_allele,allele
i64,str,str
6918,"""GG""","""GT"""
6946,"""GG""","""GT"""
17263,"""AA""","""CC"""
21665,"""AA""","""AG"""
21680,"""TT""","""TG"""
78832,"""GG""","""GA"""
129602,"""CC""","""TT"""


In [29]:
sub_individual

allele,main_allele,position,individual
str,str,i64,str
"""GT""","""GG""",6918,"""B001"""
"""GT""","""GG""",6946,"""B001"""
"""CC""","""AA""",17263,"""B001"""
"""AG""","""AA""",21665,"""B001"""
"""TG""","""TT""",21680,"""B001"""
"""GA""","""GG""",78832,"""B001"""
"""TT""","""CC""",129602,"""B001"""
